In [1]:
#| default_exp bridge

In [2]:
#| export
from __future__ import annotations

# Bridge
> A bridge from back to front.  
> Helpers for easing access to the front end from the Python kernel.

Here I use pure python/IPython facilities to help us control, inspect and modify cell outputs.   
In subsequent notebooks I'll add more facilities to help us capture the current state of the notebook (21_nbstate.ipynb) and make the link bidirectional (32_bridget.ipynb).


# Prologue

In [3]:
#| export

import os
import sys
from collections import deque
from contextlib import contextmanager
from functools import cached_property
from typing import Literal
from typing import TypeAlias

import fastcore.all as FC
import traitlets as T
from fastcore.xml import to_xml
from fasthtml.core import FT
from fasthtml.xtend import Script
from IPython.core.displayhook import CapturingDisplayHook
from IPython.core.getipython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
from IPython.display import DisplayHandle
from IPython.display import HTML
from IPython.utils.capture import CapturedIO
from olio.common import shortens
from olio.common import update_
from traitlets.config import SingletonConfigurable

In [4]:
os.environ['DEBUG'] = 'True'

In [5]:
#| export

from bridget.bridge_helpers import BLogger
from bridget.bridge_helpers import Brd_Mark
from bridget.bridge_helpers import BridgeWidget
from bridget.bridge_helpers import debug
from bridget.bridge_helpers import Loader
from bridget.bridge_helpers import marker_scr
from bridget.bridge_helpers import observer_scr
from bridget.bridge_helpers import ScriptsDetails
from bridget.helpers import bridge_cfg
from bridget.helpers import bundle_path
from bridget.helpers import id_gen
from bridget.helpers import skip
from bridget.widget_helpers import anysource
from bridget.widget_helpers import bundled

In [6]:
import inspect
import json
import operator
import time
from inspect import Parameter 
from pathlib import Path
from typing import Any
from typing import Callable
from typing import cast
from typing import DefaultDict
from typing import Iterable
from typing import Protocol
from typing import Self

import anywidget
import fasthtml.components as ft
from fastcore.all import L
from fastcore.all import NotStr
from fastcore.test import *
from fastcore.xml import escape
from fasthtml.components import highlight
from fasthtml.components import showtags
from fasthtml.core import FastHTML
from fasthtml.core import fh_cfg
from fasthtml.jupyter import JupyUvi
from httpx import get
from IPython.display import clear_output
from IPython.display import Javascript
from IPython.display import Markdown
from IPython.utils.capture import capture_output
from IPython.utils.capture import RichOutput
from olio.common import gets
from olio.common import setup_console
from olio.common import shorten

In [7]:
from bridget.bridge_helpers import _to_js
from bridget.bridge_helpers import get_loader
from bridget.bridge_helpers import handle_message
from bridget.bridge_helpers import LogCanvas
from bridget.bridge_helpers import notdebug
from bridget.display_helpers import BasicLogger
from bridget.display_helpers import NBLogger
from bridget.widget_helpers import cleanupwidgets
from bridget.widget_helpers import ScriptV

In [8]:
from fasthtml.components import Div, P, Pre, Text, Span, show, B, Details, Summary, Pre

----


In [9]:
#| exporti

DEBUG = os.environ.get('DEBUG', None) == 'True'
BUNDLE_PATH = bundle_path(__name__)
new_id = id_gen()
_n = '\n'

In [10]:
console, cprint = setup_console(140)
IDISPLAY = display

In [11]:
# needed for vfile:
%load_ext anywidget

In [12]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

# Bridge scripts

`HTMX` and other useful JS libraries.

In [13]:
#| export

def _bridge_scripts_extra():
    from fasthtml.core import surrsrc, scopesrc
    return {'surreal': surrsrc, 'css_scope_inline': scopesrc}

def bridge_scripts(htmx=True):
    from fasthtml.core import fhjsscr
    from fasthtml.xtend import Script
    htmxsrc = Script(src=f"https://unpkg.com/htmx.org@next/dist/htmx.{'' if DEBUG else 'min.'}js")()
    return update_({'htmx': htmxsrc } if htmx else {}, fasthtml_js=fhjsscr, **_bridge_scripts_extra())

# def _load_scripts(scs):
#     display(HTML(to_xml((*(scvals := [_ for _ in scs.values()]), ScriptsDetails(scvals)))))

def show_scripts(**scs: FT): 
    display(HTML(to_xml(ScriptsDetails(shortens(map(to_xml, scs.values()), 'r', 140)))), metadata=skip())

In [14]:
show_scripts(**bridge_scripts())

In [15]:
# loader = get_loader()
loader = await Loader.create()

In [16]:
loader.load_links(**bridge_scripts())

In [17]:
loader.close()
Loader.clear_instance()

# BridgeBase

In [18]:
@FC.delegates()
class BridgeBase(Loader):
    logger: BLogger
    lnks = {**bridge_scripts(), 'observer_scr':observer_scr, 'brd_mark': marker_scr}

@FC.delegates(Loader.__init__)  # type: ignore
async def get_bridge(*args, logger=None, show_scr=True, **kwargs): 
    if logger: logger.display()
    brd = await BridgeBase.create(*args, logger=logger, **kwargs)
    if show_scr: show_scripts(**brd.lnks)
    return brd

In [19]:
try: cleanupwidgets(brd)  # type: ignore
except NameError: pass
BridgeBase.clear_instance()

brd = await get_bridge()

In [20]:
brd.loaded(), brd.loaded('htmx'), brd.loaded('nah')

(True, False, False)

In [21]:
html = '''
<div>💩 👻 No style.</div>

<div>
    <style> /* Simple example. */
        me { margin: 20px; }
        me div { font-size: 5rem; }
    </style>
    <div>👻</div>
</div>
'''

display(HTML(html))

Bridge automatically loads some JavaScript libraries:

1. HTMX
2. FastHTML core scripts
3. Awesome gnat's Scope and Surreal scripts


# FastHTML Bridge display

1. This functionality patches FastHTML's IPython display system
3. This implementation:
   - Overrides all FT IPython MIME methods
   - Uses `bridge_cfg.auto_show` for display opt-in

This is a convenience and not core of this proof-of-concept notebooks.

In [22]:
fh_cfg['auto_id'] = False
bridge_cfg.auto_show=False

In [23]:
d = Div(style='color: red;', hx_trigger='click')(Text('Hi!'))
print(f"d: {d}")
print(f"showtags(d): {showtags(d)}")
display(Markdown(highlight(d)))
show(d)

d: div((text(('Hi!',),{}),),{'hx-trigger': 'click', 'style': 'color: red;'})
showtags(d): <code><pre>
&lt;div hx-trigger=&quot;click&quot; style=&quot;color: red;&quot;&gt;
&lt;text&gt;Hi!&lt;/text&gt;&lt;/div&gt;

</code></pre>


```html
<div hx-trigger="click" style="color: red;">
<text>Hi!</text></div>

```

In [24]:
Div("I'm a Div!")

```html
<div>I&#x27;m a Div!</div>

```

In [25]:
#| exporti

@FC.patch
def _repr_mimebundle_(self: FT, include=None, exclude=None):
    mb = {'text/plain': repr(self)}
    if bridge_cfg.auto_show: mb['text/html'] = self.__html__()
    else: mb['text/markdown'] = self._repr_markdown_()
    return mb

In [26]:
# #| exporti

# @FC.patch
# def _ipython_display_(self: FT):
#     IDISPLAY(self._repr_mimebundle_(), raw=True)

In [27]:
Div('me too!')

```html
<div>me too!</div>

```

In [28]:
with bridge_cfg(auto_show=True):
    display(Div("But I'm prettier!"))

div(("But I'm prettier!",),{})

In [29]:
Div('back to tags!')

```html
<div>back to tags!</div>

```

In [30]:
bridge_cfg.auto_show = True
ft.Details(open=True)(ft.Summary('dddd'), ft.Pre('eeee'))

details((summary(('dddd',),{}), pre(('eeee',),{})),{'open': True})

In [31]:
with bridge_cfg(auto_show=False): display(ft.Button('ffff'))
ft.Button('ffff')

```html
<button>ffff</button>
```

button(('ffff',),{})

----

# CellExecInfo
> IPython cell execution info.

Simple IPython event callback that captures cell id and source code of last run cell.

In [32]:
__cellid__ = None
__cellsource__ = None

In [33]:
#| export

class CellExecInfo:
    def __init__(self): 
        self._active, self.last_info, self.last_result = False,{}, {}
        self._evts = get_ipython().events  # type: ignore
    def pre_run_cell(self, info): 
        if not self._active: 
            self._evts.register('post_run_cell', self.post_run_cell)
            self._active = True
        self.last_info = {'raw_cell': info.raw_cell, 'cell_id': info.cell_id}
        if (shell := get_ipython()): 
            shell.user_ns['__cellid__'] = info.cell_id
            shell.user_ns['__cellsource__'] = info.raw_cell
    def post_run_cell(self, result):
        if not self._active: return
        self.last_result = {
            'execution_count': result.execution_count,
            'error_before_exec': result.error_before_exec,
            'error_in_exec': result.error_in_exec,
            'info': {'raw_cell': result.info.raw_cell, 'cell_id': result.info.cell_id},
            'result': result.result
        }
    def start(self): self._evts.register('pre_run_cell', self.pre_run_cell)
    def stop(self):
        if self._active:
            self._active = False
            if self.last_result: self.last_info = self.last_result['info']
            self._evts.unregister('pre_run_cell', self.pre_run_cell)
            self._evts.unregister('post_run_cell', self.post_run_cell)

In [34]:
try: csi.stop()  # type: ignore
except Exception: pass
csi = CellExecInfo()

csi.start()
test_eq(csi.last_info, {})

In [35]:
print(__cellid__)
test_eq(csi.last_result, {})
test_eq(csi.last_info['raw_cell'][:17], 'print(__cellid__)')

vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#X62sZmlsZQ%3D%3D


In [36]:
csi.stop()
test_eq(csi.last_result['info']['raw_cell'][:17], 'print(__cellid__)')

In [37]:
test_eq(csi.last_info, csi.last_result['info'])
cprint(csi.last_info, csi.last_result)

{
    'raw_cell': "print(__cellid__)\ntest_eq(csi.last_result, {})\ntest_eq(csi.last_info['raw_cell'][:17], 'print(__cellid__)')",
    'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#X62sZmlsZQ%3D%3D'
}
{
    'execution_count': 35,
    'error_before_exec': None,
    'error_in_exec': None,
    'info': {
        'raw_cell': "print(__cellid__)\ntest_eq(csi.last_result, {})\ntest_eq(csi.last_info['raw_cell'][:17], 'print(__cellid__)')",
        'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#X62sZmlsZQ%3D%3D'
    },
    'result': None
}

# BridgeBase

In [38]:
@FC.delegates()
class BridgeBase(Loader):
	logger: BLogger
	lnks = {**bridge_scripts(), 'observer_scr':observer_scr, 'brd_mark': marker_scr}

	def __init__(self, *args, **kwargs):
		self.csi = CellExecInfo()
		super().__init__(*args, **kwargs)

	def close(self):
		self.csi.stop()
		super().close()
	def __del__(self): self.close()

	def on_info(self, *args, info:str, **kwargs):
		super().on_info(*args, info=info, **kwargs)
		if info == 'loaded': self.csi.start()

@FC.delegates(Loader.__init__)  # type: ignore
async def get_bridge(*args, logger=None, show_scr=True, **kwargs): 
    if logger: logger.display()
    brd = await BridgeBase.create(*args, logger=logger, **kwargs)
    if show_scr: show_scripts(**brd.lnks)
    return brd

In [39]:
cleanupwidgets(brd)
type(brd).clear_instance()

bridge_cfg.auto_id = True
brd = await get_bridge()

In [40]:
display(brd.csi.last_info['cell_id'])

'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y101sZmlsZQ%3D%3D'

In [41]:
print(10)
li = brd.csi.last_info

10


In [42]:
cprint(li)
cprint(brd.csi.last_info)

{
    'raw_cell': 'print(10)\nli = brd.csi.last_info',
    'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y102sZmlsZQ%3D%3D'
}

{
    'raw_cell': 'cprint(li)\ncprint(brd.csi.last_info)',
    'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y103sZmlsZQ%3D%3D'
}

In [43]:
brd.close()

# IPython display_pub hook
> `display_pub` hook for `display_id` and `brd-mark`

This will add bridge metadata to the display message. This will be handy to target specific cells and when we can capture the notebook state down the road.

In [44]:
#| export

class Bridged:
    def __init__(self, 
            dhs: deque[DisplayHandle]|None = None, msgs: list[dict]|None = None, register: bool = False):
        self.dhs = deque(maxlen=100) if dhs is None else dhs
        if DEBUG: self.msgs = [] if msgs is None else msgs
        if register: self.register()
    
    def register(self):
        shell = get_ipython()
        if shell and (reg := getattr(shell.display_pub, 'register_hook', None)): return reg(self.bridged)
        return False

    def unregister(self):
        shell = get_ipython()
        if shell and (unreg := getattr(shell.display_pub, 'unregister_hook', None)): return unreg(self.bridged)
        return False
    
    def __del__(self): self.unregister()
    
    def bridged(self, msg):
        "Augment display messages with bridge stuff."
        if DEBUG: self.msgs.append(msg)
        if msg['msg_type'] in ('display_data', 'update_display_data'):
            # hoist display_id or brd_did to output metadata
            cnt, md, did = msg['content'], msg['metadata'], None
            cmd = cnt['metadata']
            if (brdmd := cmd.get('bridge', None)) and brdmd.get('skip', None): return msg
            if 'text/html' not in (d := cnt['data']): return msg
            if trn := cnt['transient']: did = trn['display_id']
            elif cmd:
                for k,v in cmd.items():
                    if brd_id := v.get('brd_did', None): did = brd_id; break
            if not did and bridge_cfg.auto_id: did = new_id()
            if did: 
                if 'display_id' not in trn: trn['display_id'] = did
                if not self.dhs or (did != self.dhs[-1].display_id):
                    self.dhs.append(DisplayHandle(display_id=did))
                # cmd['brd_did'] = did
                # md['bridge'] = {'id': did}
                # add brd-mark to output
                if 'text/html' in (d := cnt['data']):
                    d['text/html'] = d['text/html'] + _n + to_xml(Brd_Mark(id=did))
        return msg


In [45]:
try: brdd.unregister()  # type: ignore
except Exception: pass

brdd = Bridged(register=True)

### set display_id with HTML metadata

```json
{
    ...,
    'msg_type': 'display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>', 
            'text/html': "<div>I'm marked!... MAAARKED!!</div>"
        },
        'metadata': {'text/html': {'brd_did': 'b8b568b9a-c02e1576-c3a3c120-167cedda'}},
        'transient': {}
    },
    'metadata': {}
}

{
    ...,
    'msg_type': 'display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>',
            'text/html': '<div>I\'m marked!... MAAARKED!!</div><brd-mark id="b8b568b9a-c02e1576-c3a3c120-167cedda"></brd-mark>'
        },
        'metadata': {'text/html': {'brd_did': 'b8b568b9a-c02e1576-c3a3c120-167cedda'}},
        'transient': {'display_id': 'b8b568b9a-c02e1576-c3a3c120-167cedda'}
    },
    'metadata': {}
}
```

In [46]:
did = new_id()
print(f"{did=}")
display(HTML("<div>I'm marked!... MAAARKED!!</div>", metadata={'brd_did': did}))

did='ba4f47090-9a07a612-626cda36-212e6fd2'


In [47]:
dh = brdd.dhs[-1]
test_eq(dh.display_id, did)

Dor convenience, `Bridged` stores in `dhs` the last display handles used.  

Note however that there's no mapping between display handles and the objects displayed or the cells that displayed them. We have to wait a bit to get that functionality.


```json
{
    ...,
    'msg_type': 'update_display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>', 
            'text/html': "<div>I'm doomed!... DOOOOOMED!!</div>"},
        'metadata': {},
        'transient': {'display_id': 'b5c00d851-9da4a95e-36473c24-3d04534d'}
    },
    'metadata': {}
}

{
    ...,
    'msg_type': 'update_display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>',
            'text/html': '<div>I\'m doomed!... DOOOOOMED!!</div><brd-mark id="b5c00d851-9da4a95e-36473c24-3d04534d"></brd-mark>'
        },
        'metadata': {},
        'transient': {'display_id': 'b5c00d851-9da4a95e-36473c24-3d04534d'}
    },
    'metadata': {}
}
```

In [48]:
dh.update(HTML("<div>I'm doomed!... DOOOOOMED!!</div>"))

## set display_id with display(...) kwarg

```json
{
    ...,
    'msg_type': 'display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>', 
            'text/html': "<div>I'm marked!... MAAARKED!!</div>"
        },
        'metadata': {},
        'transient': {'display_id': '2307db4acc4fda0ba305ffdda518748a'}
    },
    'metadata': {}
}

{
    ...,
    'msg_type': 'display_data',
.    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>',
            'text/html': '<div>I\'m marked!... MAAARKED!!</div><brd-mark id="2307db4acc4fda0ba305ffdda518748a"></brd-mark>'
        },
        'metadata': {},
        'transient': {'display_id': '2307db4acc4fda0ba305ffdda518748a'}
    },
    'metadata': {}
}
```


In [49]:
dh = display(HTML("<div>I'm marked!... MAAARKED!!</div>"), display_id=True)

```json
{
    ...,
    'msg_type': 'update_display_data',
    'content': {
        'data': {'text/plain': '<IPython.core.display.HTML object>', 'text/html': "<div>I'm doomed!... DOOOOOMED!!</div>"},
        'metadata': {},
        'transient': {'display_id': 'c3d21633d341d2463f13ef40730e8c4a'}
    },
    'metadata': {}
}

{
    ...,
    'msg_type': 'update_display_data',
    'content': {
        'data': {
            'text/plain': '<IPython.core.display.HTML object>',
            'text/html': '<div>I\'m doomed!... DOOOOOMED!!</div><brd-mark id="c3d21633d341d2463f13ef40730e8c4a"></brd-mark>'
        },
        'metadata': {},
        'transient': {'display_id': 'c3d21633d341d2463f13ef40730e8c4a'}
    },
    'metadata': {}
}
```


In [50]:
if dh: dh.update(HTML("<div>I'm doomed!... DOOOOOMED!!</div>"))

## Multi objects display

In [51]:
dh = display(HTML("<div>Multi 1</div>"), HTML("<div>Multi 2</div>"), display_id=True)

In [52]:
if dh: dh.update(HTML("<div>Multi 3</div>"))

IPython display assign the same display id to each object. But the front end will only register the last object displayed.  
I don't know if this is intended, a bug, or a VSCode foible (VScode clearly separates every display, unlike Jupyter).

In [53]:
brdd.dhs.clear()

In [54]:
display(
    HTML("<div>Multi 1</div>", metadata={'brd_did': (did1 := new_id())}), 
    HTML("<div>Multi 2</div>", metadata={'brd_did': (did2 := new_id())})
)

In [55]:
dh1, dh2 = brdd.dhs[-2], brdd.dhs[-1]
dh1.update(HTML("<div>Multi 3</div>"))

time.sleep(0.01)

dh2.update(HTML("<div>Multi 4</div>"))

We can somewhat sidestep this by using Bridge metadata.

In [56]:
bridge_cfg.auto_id = True

display(HTML("<div>I'm auto-id'd--</div>"))

In [57]:
brdd.dhs[-1].update(HTML("<div>--as shown above.</div>"))

if `bridge_cfg.auto_id` is True, there's no need to use bridge metadata. Every `display`ed `HTML` object will receive an auto-generated display id.


In [58]:
brdd.dhs.clear()

In [59]:
lggr = BasicLogger().setup(height=400)
lggr.log('A dark and stormy night...')
lggr.log(str(vars(bridge_cfg)))

In [60]:
test_eq(len(brdd.dhs), 0)

In [61]:
brdd.unregister()

True

----

In [62]:
def show_msgs(brdd: Bridged):
    for msg in brdd.msgs:
        d = msg.copy()
        # d['parent_header'] = {'...': '...'}
        # d['header'] = {'...': '...'}
        del d['parent_header']
        del d['header']
        del d['tracker']
        del d['msg_id']
        if not d['metadata']: del d['metadata']
        try: del d['content']['data']['text/plain']
        except: pass
        if h := d['content']['data'].get('text/html', None): d['content']['data']['text/html'] = shorten(h, 'r', 120)
        cprint(d)

show_msgs(brdd)

{
    'msg_type': 'display_data',
    'content': {
        'data': {'text/html': '<div>I\'m marked!... MAAARKED!!</div>\n<brd-mark id="ba4f47090-9a07a612-626cda36-212e6fd2"></brd-mark>'},
        'metadata': {'text/html': {'brd_did': 'ba4f47090-9a07a612-626cda36-212e6fd2'}},
        'transient': {'display_id': 'ba4f47090-9a07a612-626cda36-212e6fd2'}
    }
}

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {'text/html': '<div>I\'m doomed!... DOOOOOMED!!</div>\n<brd-mark id="ba4f47090-9a07a612-626cda36-212e6fd2"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': 'ba4f47090-9a07a612-626cda36-212e6fd2'}
    }
}

{
    'msg_type': 'display_data',
    'content': {
        'data': {'text/html': '<div>I\'m marked!... MAAARKED!!</div>\n<brd-mark id="4fc4ca1e1b85ecce61090bde4c767382"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': '4fc4ca1e1b85ecce61090bde4c767382'}
    }
}

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {'text/html': '<div>I\'m doomed!... DOOOOOMED!!</div>\n<brd-mark id="4fc4ca1e1b85ecce61090bde4c767382"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': '4fc4ca1e1b85ecce61090bde4c767382'}
    }
}

{
    'msg_type': 'display_data',
    'content': {
        'data': {'text/html': '<div>Multi 1</div>\n<brd-mark id="90e6c76741b469930ca4a9a4a92f333f"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': '90e6c76741b469930ca4a9a4a92f333f'}
    }
}

...

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {'text/html': '<div>--as shown above.</div>\n<brd-mark id="b84e13ed2-913592e4-cb627b42-7fd1fc40"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': 'b84e13ed2-913592e4-cb627b42-7fd1fc40'}
    }
}

{
    'msg_type': 'display_data',
    'content': {'data': {}, 'metadata': {'bridge': {'skip': True}}, 'transient': {'display_id': '7a68aa11cb1c72f55c64bd6e0df3a53b'}}
}

{
    'msg_type': 'display_data',
    'content': {'data': {}, 'metadata': {'bridge': {'skip': True}}, 'transient': {'display_id': '2af03e851a041dd154fe6aef1ca46f5d'}}
}

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {
            'text/html': "<div style='max-height: 400px; border: 1px solid black; overflow: auto;'><div>A dark and stormy 
night...</div></div>"
        },
        'metadata': {'bridge': {'skip': True}},
        'transient': {'display_id': '2af03e851a041dd154fe6aef1ca46f5d'}
    }
}

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {
            'text/html': "<div style='max-height: 400px; border: 1px solid black; overflow: auto;'><div>{'auto_show': True, 'auto_id': 
True}</div>…"
        },
        'metadata': {'bridge': {'skip': True}},
        'transient': {'display_id': '2af03e851a041dd154fe6aef1ca46f5d'}
    }
}

# BridgeBase

In [63]:
@FC.delegates()
class BridgeBase(Loader):
	logger: BLogger
	lnks = {**bridge_scripts(), 'observer_scr':observer_scr, 'brd_mark': marker_scr}
	dhs: deque[DisplayHandle]

	def __init__(self, *args, **kwargs):
		if DEBUG: self.msgs = []
		self.csi, self.dhs, self.hook = CellExecInfo(), deque(maxlen=100), None
		super().__init__(*args, **kwargs)

	def close(self):
		self.csi.stop()
		if self.hook: self.hook.unregister()
		super().close()
	def __del__(self): self.close()

	def on_info(self, *args, info:str, **kwargs):
		super().on_info(*args, info=info, **kwargs)
		if info == 'loaded': # now we can initialize the scripts
			if DEBUG: show_scripts(**self.lnks)
			self.hook = Bridged(self.dhs, self.msgs if DEBUG else None, register=True)
			self.csi.start()


@FC.delegates(Loader.__init__)  # type: ignore
async def get_bridge(*args, logger=None, show_scr=False, **kwargs): 
    if logger: logger.display()
    brd = await BridgeBase.create(*args, logger=logger, **kwargs)
    if show_scr: show_scripts(**brd.lnks)
    return brd

In [64]:
cleanupwidgets(brd)
type(brd).clear_instance()

bridge_cfg.auto_id = True
brd = await get_bridge()

In [65]:
display(brd.csi.last_info['cell_id'])

'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y155sZmlsZQ%3D%3D'

In [66]:
brd.close()

In [67]:
show_msgs(brd.hook)  # type: ignore

{'msg_type': 'display_data', 'content': {'data': {}, 'metadata': {}, 'transient': {}}}

# OutputCapture

Our goal is to control (for now) all HTML output. Bridge can now set metadata of any display message, those that go through display_pub. Bridge capture all `display`, direct or FastHTML bridge.  
But there're other means to output HTML that doesn't follow the display_pub path: auto display of cell's final expression. That goes through another path, display_hook.

In [68]:
#| export

class OutputCapture(SingletonConfigurable):
    shell: InteractiveShell
    def __init__(self):
        super().__init__()
        self.shell = get_ipython()  # type: ignore
        if DEBUG: self._cells, self._outputs, self._captured = deque(maxlen=100), deque(maxlen=100), deque(maxlen=100)
        # if DEBUG: self._exres = deque(maxlen=100)
        self._active, self.run_outputs = False, []
        self.displayhook = CapturingDisplayHook(shell=self.shell, outputs=self.run_outputs)
    
    @contextmanager
    def _capture(self):
        self.run_outputs.clear()
        try: 
            save_display_hook, sys.displayhook = sys.displayhook, self.displayhook
            self._active = True
            yield CapturedIO(stdout=None, stderr=None, outputs=self.run_outputs)
        finally: 
            self._active = False
            sys.displayhook = save_display_hook
    
    def __call__(self, cell):
        if DEBUG: self._cells.append(cell)
        with self._capture() as io:
            # result = self.shell.run_cell(cell, silent=True, cell_id=self.shell.user_ns.get('__cellid__'))
            self.shell.run_cell(cell, cell_id=self.shell.user_ns.get('__cellid__'))
        # self._exres.append(result)
        # assert len(io._outputs) == 0, "Outputs should be 0."
        if io._outputs: 
            if DEBUG: self._outputs.extend(io._outputs)
            assert len(io._outputs) <= 1, "Only one output is supported"
            display(io.outputs[-1])
    
    def _transform(self, lines):
        "Input transformer function"
        if not lines or self._active: return lines
        if lines[0].startswith('import debugpy;debugpy.listen('): return lines
        if lines[0].startswith('import debugpy\ndebugpy.debug_this_thread()'): return lines
        return ['get_capturer()(%r)\n' % ''.join(lines)]
            
    def register(self):
        self.shell.user_ns['get_capturer'] = get_capturer
        if DEBUG: self._cells, self._outputs = deque(maxlen=100), deque(maxlen=100)
        # self.shell.input_transformers_cleanup.append(self._transform)
        self.shell.input_transformer_manager.line_transforms.append(self._transform)
        
    def unregister(self):
        try: 
            # self.shell.input_transformers_cleanup.remove(self._transform)
            self.shell.input_transformer_manager.line_transforms.remove(self._transform)
        except (ValueError, NameError): pass
    
    def __del__(self): self.unregister()


def get_capturer(): return OutputCapture.instance()

In [69]:
cleanupwidgets(brd)  # type: ignore
type(brd).clear_instance()

bridge_cfg.auto_id = True
brd = await get_bridge()

In [70]:
get_capturer().unregister()
OutputCapture.clear_instance()

get_capturer().register()

In [71]:
brd.dhs.clear()
1+3

4

In [72]:
if DEBUG: test_eq(get_capturer()._outputs[-1], {'data': {'text/plain': '4'}, 'metadata': {}})
test_eq(len(brd.dhs), 0)

In [73]:
HTML('<div>asdf</div>')

In [74]:
if DEBUG: test_eq(get_capturer()._outputs[-1]['data']['text/plain'], '<IPython.core.display.HTML object>')
test_eq(len(brd.dhs), 1)

In [75]:
get_capturer().unregister()

In [76]:
brd.dhs[-1].update(HTML('<div>qwer</div>'))

In [77]:
brd.close()

In [78]:
brd.csi.last_info, brd.csi.last_result

({'raw_cell': "brd.dhs[-1].update(HTML('<div>qwer</div>'))",
  'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y203sZmlsZQ%3D%3D'},
 {'execution_count': 76,
  'error_before_exec': None,
  'error_in_exec': None,
  'info': {'raw_cell': "brd.dhs[-1].update(HTML('<div>qwer</div>'))",
   'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y203sZmlsZQ%3D%3D'},
  'result': None})

----

In [79]:
def show_outputs():
    oo = get_capturer()._outputs
    for o in oo:
        cprint(o)

def show_cells():
    oo = get_capturer()._cells
    for o in oo:
        cprint(o)

show_outputs()
show_cells()


{'data': {'text/plain': '4'}, 'metadata': {}}

{
    'data': {
        'text/plain': '<IPython.core.display.HTML object>',
        'text/html': '<div>asdf</div>\n<brd-mark id="b1587a51e-0388d52e-60c80df0-7cb04aa6"></brd-mark>'
    },
    'metadata': {}
}

brd.dhs.clear()
1+3

if DEBUG: test_eq(get_capturer()._outputs[-1], {'data': {'text/plain': '4'}, 'metadata': {}})
test_eq(len(brd.dhs), 0)

HTML('<div>asdf</div>')

if DEBUG: test_eq(get_capturer()._outputs[-1]['data']['text/plain'], '<IPython.core.display.HTML object>')
test_eq(len(brd.dhs), 1)

get_capturer().unregister()

In [80]:
show_msgs(brd.hook)  # type: ignore

{'msg_type': 'display_data', 'content': {'data': {}, 'metadata': {}, 'transient': {}}}

{
    'msg_type': 'display_data',
    'content': {
        'data': {'text/html': '<div>asdf</div>\n<brd-mark id="b1587a51e-0388d52e-60c80df0-7cb04aa6"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': 'b1587a51e-0388d52e-60c80df0-7cb04aa6'}
    }
}

{
    'msg_type': 'update_display_data',
    'content': {
        'data': {'text/html': '<div>qwer</div>\n<brd-mark id="b1587a51e-0388d52e-60c80df0-7cb04aa6"></brd-mark>'},
        'metadata': {},
        'transient': {'display_id': 'b1587a51e-0388d52e-60c80df0-7cb04aa6'}
    }
}

----

# BridgeBase

In [81]:
@FC.delegates()
class BridgeBase(Loader):
	logger: BLogger
	lnks = {**bridge_scripts(), 'observer_scr':observer_scr, 'brd_mark': marker_scr}
	dhs: deque[DisplayHandle]

	def __init__(self, *args, **kwargs):
		if DEBUG: self.msgs = []
		self.csi, self.dhs, self.hook = CellExecInfo(), deque(maxlen=100), None
		super().__init__(*args, **kwargs)

	def close(self):
		self.csi.stop()
		get_capturer().unregister()
		if self.hook: self.hook.unregister()
		super().close()
	def __del__(self): self.close()

	def on_info(self, *args, info:str, **kwargs):
		super().on_info(*args, info=info, **kwargs)
		if info == 'loaded':
			self.hook = Bridged(self.dhs, self.msgs if DEBUG else None, register=True)
			get_capturer().register()
			self.csi.start()


@FC.delegates(Loader.__init__)  # type: ignore
async def get_bridge(*args, logger=None, show_scr=False, **kwargs): 
    if logger: logger.display()
    brd = await BridgeBase.create(*args, logger=logger, **kwargs)
    if show_scr: show_scripts(**brd.lnks)
    return brd

In [82]:
cleanupwidgets(brd)  # type: ignore
get_capturer().unregister()
OutputCapture.clear_instance()
type(brd).clear_instance()

bridge_cfg.auto_id = True
brd = await get_bridge()

In [83]:
display(__cellid__)#brd.csi.last_info['cell_id'])

'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y216sZmlsZQ%3D%3D'

In [84]:
brd.csi.last_info, brd.csi.last_result

({'raw_cell': 'brd.csi.last_info, brd.csi.last_result\n',
  'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y220sZmlsZQ%3D%3D'},
 {'execution_count': 83,
  'error_before_exec': None,
  'error_in_exec': None,
  'info': {'raw_cell': "display(__cellid__)#brd.csi.last_info['cell_id'])",
   'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y216sZmlsZQ%3D%3D'},
  'result': None})

# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```


In [85]:
md_samp = '''
# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```
'''

Markdown(md_samp)


# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```


In [86]:
# %%HTML
# <pre>aaaa</pre>
# <script type="module">
# debugger;
# import ZeroMd from 'https://cdn.jsdelivr.net/npm/zero-md@3'
# customElements.define('zero-md', ZeroMd)
# </script>

zeromd_js = '''
import ZeroMd from 'https://cdn.jsdelivr.net/npm/zero-md@3'
customElements.define('zero-md', ZeroMd);
'''

zeromd_scr = Script(debug(zeromd_js), id='zeromd', type='module')

In [87]:
brd.load_links(zeromd=zeromd_scr)

In [88]:
from fasthtml.components import Zero_md, Template
from bridget.widget_helpers import StyleV

def render_local_md(md, css=''):
    css_template = Template(StyleV(css), data_append=True)
    return Zero_md(css_template, Script(md, type="text/markdown"))

In [89]:
md_ft = render_local_md(md_samp)

# with bridge_cfg(auto_show=False):
#     display(md_ft)

md_ft

zero-md((template((style(('',),{}),),{'data-append': True}), script(('\n# &lt;zero-md&gt;\n\n> Ridiculously simple zero-config markdown displayer\n\nA vanilla markdown-to-html web component based on\n[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external\nMD file.\n\nFeaturing:\n\n- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)\n- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams\n- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)\n\n```python\na = {\'a\': \'asdf\', \'b\': 10}\nprint("local server is listening on port 8080")\n```\n',),{'type': 'text/markdown'})),{})

In [90]:
cprint(to_xml(md_ft))

<zero-md><template data-append>    <style></style>
</template><script type="text/markdown">
# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

-  Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
-  [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
-  Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```
</script></zero-md>

In [91]:
brd.close()

# HTMX Commander
> Python wrapper of HTMX API.

Currently only implemented `htmx.swap()` manually. If useful, I'll automate wrappers generation from htmx docs.


In [92]:
#| export

commander_js = BUNDLE_PATH / 'js/commander.js'
commander_esm = bundled(commander_js, bundle=__name__, bundler='copy')
commander_scr = Script(debug(commander_esm.source), id='htmx-commander')

class HTMXCommander(BridgeWidget):
    _esm = anysource(commander_esm(debugger=DEBUG), '''
export default { 
    async initialize({ model }) {
        await initializeCommander(model.get('output_sels'));
        model.on("msg:custom", on_commander_msg);
        model.send({ ctx: 'commander', kind: 'info', info: 'loaded' });
    }
};
''')
    
    output_sels = T.List(['.output', '.jp-Cell-outputArea']).tag(sync=True)

    ctx_name = T.Unicode('commander').tag(sync=True)

    def swap(self: HTMXCommander,
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin','beforeend','afterend','delete','none'],
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ): ...

In [93]:
cleanupwidgets('cmdr')

cmdr = HTMXCommander(show=True)

## API

### Method - `htmx.swap()` {#swap}

Performs swapping (and settling) of HTML content

##### Parameters

* `target` - the HTML element or string selector of swap target
* `content` - string representation of content to be swapped
* `swapSpec` - swapping specification, representing parameters from `hx-swap`
  * `swapStyle` (required) - swapping style (`innerHTML`, `outerHTML`, `beforebegin` etc)
  * `swapDelay`, `settleDelay` (number) - delays before swapping and settling respectively
  * `transition` (bool) - whether to use HTML transitions for swap
  * `ignoreTitle` (bool) - disables page title updates
  * `head` (string) - specifies `head` tag handling strategy (`merge` or `append`). Leave empty to disable head handling
  * `scroll`, `scrollTarget`, `show`, `showTarget`, `focusScroll` - specifies scroll handling after swap
* `swapOptions` - additional *optional* parameters for swapping
  * `select` - selector for the content to be swapped (equivalent of `hx-select`)
  * `selectOOB` - selector for the content to be swapped out-of-band (equivalent of `hx-select-oob`)
  * `eventInfo` - an object to be attached to `htmx:afterSwap` and `htmx:afterSettle` elements
  * `anchor` - an anchor element that triggered scroll, will be scrolled into view on settle. Provides simple alternative to full scroll handling
  * `contextElement` - DOM element that serves as context to swapping operation. Currently used to find extensions enabled for specific element
  * `afterSwapCallback`, `afterSettleCallback` - callback functions called after swap and settle respectively. Take no arguments


##### Example

```js
    // swap #output element inner HTML with div element with "Swapped!" text
    htmx.swap("#output", "<div>Swapped!</div>", {swapStyle: 'innerHTML'});
```


In [94]:
#| export

SwapStyleT: TypeAlias = Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin',
                        'beforeend','afterend','delete','none']

In [95]:
#| export

def swap(self, 
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: SwapStyleT='innerHTML',
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ):
    d = {
        'target': target,
        'content': content,
        'swapSpec': update_(**{
            'swapStyle': swapStyle, 'swapDelay': swapDelay, 'settleDelay': settleDelay,
            'transition': transition,
            # 'ignoreTitle': ignoreTitle, 'head': head,
            'scroll': scroll, 'scrollTarget': scrollTarget,
            'show': show, 'showTarget': showTarget, 'focusScroll': focusScroll,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
        'swapOptions': update_(**{
            'select': select, 'selectOOB': selectOOB,
            # 'eventInfo': eventInfo,
            'anchor': anchor,
            # 'contextElement': contextElement,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
    }
    self.send({
        'cmd': 'swap',
        'args': [*d.values()]
    })

FC.patch_to(HTMXCommander)(swap)

## Test swap

In [96]:
%%HTML

<div id="output-99">Original</div>

In [97]:
cmdr.swap('#output-99', '<div>Swapped!</div>', swapStyle='innerHTML')

In [98]:
cleanupwidgets('cmdr2')

cmdr2 = HTMXCommander()

## ----
First steps exploring automation. Skip it.

In [99]:
sig = inspect.signature(cmdr.swap)
print(sig)
print(sig.parameters)
target_param = sig.parameters['swapStyle']
target_param.name, target_param.default, target_param.annotation, target_param.kind


(target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)
OrderedDict({'target': <Parameter "target">, 'content': <Parameter "content">, 'swapStyle': <Parameter "swapStyle: 'SwapStyleT' = 'innerHTML'">, 'swapDelay': <Parameter "swapDelay: 'int | None' = None">, 'settleDelay': <Parameter "settleDelay: 'int | None' = None">, 'transition': <Parameter "transition: 'bool | None' = None">, 'scroll': <Parameter "scroll: 'str | None' = None">, 'scrollTarget': <Parameter "scrollTarget: 'str | None' = None">, 'show': <Parameter "show: 'str | None' = None">, 'showTarget': <Parameter "showTarget: 'str | None' = None">, 'focusScroll': <Parameter "focusSc

('swapStyle', 'innerHTML', 'SwapStyleT', <_ParameterKind.KEYWORD_ONLY: 3>)

In [100]:

bb = sig.bind(target='#target', content='<content />', swapStyle='innerHTML', swapDelay=1000)
bb.arguments, bb.signature, bb.args, bb.kwargs


({'target': '#target',
  'content': '<content />',
  'swapStyle': 'innerHTML',
  'swapDelay': 1000},
 <Signature (target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)>,
 ('#target', '<content />'),
 {'swapStyle': 'innerHTML', 'swapDelay': 1000})

In [101]:
swap_args = {
    'target': 'target',
    'content': 'content',
    '*': '*',
    'swapSpec': {
        'swapStyle': 'swapStyle', 'swapDelay': 'swapDelay', 'settleDelay': 'settleDelay',
        'transition': 'transition',
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': 'scroll', 'scrollTarget': 'scrollTarget',
        'show': 'show', 'showTarget': 'showTarget', 'focusScroll': 'focusScroll',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}


In [102]:
from inspect import Parameter


swap_args = {
    'target': Parameter('target', Parameter.POSITIONAL_OR_KEYWORD),
    'content': Parameter('content', Parameter.POSITIONAL_OR_KEYWORD),
    'swapSpec': {
        'swapStyle': Parameter('swapStyle', Parameter.KEYWORD_ONLY, annotation=SwapStyleT), 
        'swapDelay': Parameter('swapDelay', Parameter.KEYWORD_ONLY), 
        'settleDelay': Parameter('settleDelay', Parameter.KEYWORD_ONLY),
        'transition': Parameter('transition', Parameter.KEYWORD_ONLY),
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': Parameter('scroll', Parameter.KEYWORD_ONLY), 
        'scrollTarget': Parameter('scrollTarget', Parameter.KEYWORD_ONLY),
        'show': Parameter('show', Parameter.KEYWORD_ONLY), 
        'showTarget': Parameter('showTarget', Parameter.KEYWORD_ONLY), 
        'focusScroll': Parameter('focusScroll', Parameter.KEYWORD_ONLY),
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}


In [103]:
def build_args_dict(args_desc: dict, *args, **kwargs) -> dict:
    "Builds a dictionary from args/kwargs based on an arguments descriptor"
    result = {}
    
    # Process descriptor items in order
    for k, v in args_desc.items():
        if isinstance(v, dict):
            # Handle nested dictionary descriptors
            nested = {
                v2: kwargs.get(k2) 
                for k2, v2 in v.items() 
                if kwargs.get(k2) is not None
            }
            if nested: result[k] = nested
        else:
            # For non-dict values, first try kwargs, then fallback to positional args
            if k in kwargs:
                result[v] = kwargs[k]
            elif len(args) > 0:
                result[v] = args[0]
                args = args[1:]
    
    return result


build_args_dict(swap_args, '#target', '<content />', swapStyle='innerHTML', swapDelay=1000)



{<Parameter "target">: '#target',
 <Parameter "content">: '<content />',
 'swapSpec': {<Parameter "swapStyle: Literal['innerHTML', 'outerHTML', 'testContent', 'beforebegin', 'afterbegin', 'beforeend', 'afterend', 'delete', 'none']">: 'innerHTML',
  <Parameter "swapDelay">: 1000}}

# BridgeBase

In [104]:
#| export

@FC.delegates()
class BridgeBase(Loader):
    logger: BLogger
    lnks = {**bridge_scripts(), 'observer_scr':observer_scr, 'brd_mark': marker_scr}
    dhs: deque[DisplayHandle]
    # commander: HTMXCommander | None = None

    def __init__(self, *args, 
            # companions_loader: Callable[[Self], list[BridgeWidget]]|None = None,
            **kwargs
        ):
        if DEBUG: self.msgs = []
        self.csi, self.dhs, self.hook = CellExecInfo(), deque(maxlen=100), None
        # self.companions = []
        # self._companions_loader = companions_loader
        super().__init__(*args, **kwargs)

    def close(self):
        self.csi.stop()
        get_capturer().unregister()
        if self.hook: self.hook.unregister()
        # map(lambda c: c.close(), self.companions)
        super().close()
    def __del__(self): self.close()

    def on_info(self, *args, info:str, **kwargs):
        super().on_info(*args, info=info, **kwargs)
        if info == 'loaded': # now we can initialize the scripts
            self.hook = Bridged(self.dhs, self.msgs if DEBUG else None, register=True)
            get_capturer().register()
            self.csi.start()

    def on_loadLinks(self, *args, success:list[str], **kwargs):
        super().on_loadLinks(*args, success=success, **kwargs)
        # if self._companions_loader: self.companions = self._companions_loader(self)
    
    @cached_property
    def commander(self) -> HTMXCommander | None:
        if self.loaded('htmx'):
            return HTMXCommander(logger=self.logger, show=False)


@FC.delegates(Loader.__init__)  # type: ignore
async def get_bridge(*args, logger=None, show_scr=False, **kwargs): 
    if logger: logger.display()
    brd = await BridgeBase.create(*args, logger=logger, **kwargs)
    if show_scr: show_scripts(**brd.lnks)
    return brd

In [105]:
cleanupwidgets(brd)
get_capturer().unregister()
OutputCapture.clear_instance()
type(brd).clear_instance()

bridge_cfg.auto_id = True
brd = await get_bridge()

In [106]:
print(__cellid__)

vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y261sZmlsZQ%3D%3D


In [107]:
brd.close()

In [108]:
brd.csi.last_info, brd.csi.last_result

({'raw_cell': 'print(__cellid__)',
  'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y261sZmlsZQ%3D%3D'},
 {'execution_count': 106,
  'error_before_exec': None,
  'error_in_exec': None,
  'info': {'raw_cell': 'print(__cellid__)',
   'cell_id': 'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/15_bridge.ipynb#Y261sZmlsZQ%3D%3D'},
  'result': None})

----

In [109]:
#| export

shell = get_ipython()
if shell:
    shell.user_ns['__cellid__'] = None
    shell.user_ns['__cellsource__'] = None

# Colophon
----


In [110]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [111]:
if FC.IN_NOTEBOOK:
    nb_path = '15_bridge.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)